<a href="https://colab.research.google.com/github/belkisKOS/ai_model/blob/main/test_newest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Weight-Standardized Conv2d
class WSConv2d(nn.Conv2d):
    def forward(self, x):
        weight = self.weight
        mean = weight.mean(dim=[1,2,3], keepdim=True)
        std = weight.view(weight.size(0), -1).std(dim=1, keepdim=True).view(-1,1,1,1) + 1e-5
        weight = (weight - mean) / std
        return F.conv2d(x, weight, self.bias, self.stride, self.padding, self.dilation, self.groups)

# Squeeze-and-Excitation Block
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

# Bottleneck Residual Block
class BottleneckBlock(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.gn1 = nn.GroupNorm(32, in_channels)
        self.conv1 = WSConv2d(in_channels, mid_channels, kernel_size=1, bias=False)
        self.gn2 = nn.GroupNorm(32, mid_channels)
        self.conv2 = WSConv2d(mid_channels, mid_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.gn3 = nn.GroupNorm(32, mid_channels)
        self.conv3 = WSConv2d(mid_channels, out_channels, kernel_size=1, bias=False)
        self.se = SEBlock(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.gn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.gn2(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.gn3(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.se(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return out

# The full DeepResidualCNN
class DeepResidualCNN(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.stem = nn.Sequential(
            WSConv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.GroupNorm(32, 64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.layer1 = self._make_layer(64, 64, 256, blocks=3, stride=1)
        self.layer2 = self._make_layer(256, 128, 512, blocks=4, stride=2)
        self.layer3 = self._make_layer(512, 256, 1024, blocks=6, stride=2)
        self.layer4 = self._make_layer(1024, 512, 2048, blocks=3, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Linear(2048, 512),
            nn.GELU(),
            nn.LayerNorm(512),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def _make_layer(self, in_channels, mid_channels, out_channels, blocks, stride):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                WSConv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.GroupNorm(32, out_channels)
            )
        layers = [BottleneckBlock(in_channels, mid_channels, out_channels, stride, downsample)]
        for _ in range(1, blocks):
            layers.append(BottleneckBlock(out_channels, mid_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Instantiate model and print summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepResidualCNN(num_classes=3).to(device)
print(model)


DeepResidualCNN(
  (stem): Sequential(
    (0): WSConv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): GroupNorm(32, 64, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): BottleneckBlock(
      (gn1): GroupNorm(32, 64, eps=1e-05, affine=True)
      (conv1): WSConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
      (conv2): WSConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
      (conv3): WSConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (se): SEBlock(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=256, out_features=16, bias=False)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=1

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepResidualCNN(num_classes=3).to(device)

model.load_state_dict(torch.load("/content/drive/MyDrive/best_newest2_model.pth"))
model.eval()  # set model to evaluation mode


DeepResidualCNN(
  (stem): Sequential(
    (0): WSConv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): GroupNorm(32, 64, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): BottleneckBlock(
      (gn1): GroupNorm(32, 64, eps=1e-05, affine=True)
      (conv1): WSConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
      (conv2): WSConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
      (conv3): WSConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (se): SEBlock(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=256, out_features=16, bias=False)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=1

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

# Define grayscale loader to match training preprocessing
def pil_loader_gray(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('L')

transform = transforms.Compose([
    transforms.ToTensor(),
])

test_dir = "/content/drive/MyDrive/Atelier2025_SCI_1st_Dataset/processed_dataset2/test"

test_dataset = datasets.ImageFolder(root=test_dir, transform=transform, loader=pil_loader_gray)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)


In [ ]:
criterion = torch.nn.CrossEntropyLoss()  # define the loss function

model.eval()
test_loss = 0
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)

        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_loss = test_loss / total
test_accuracy = correct / total

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Also print detailed classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))
